In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [4]:
from collections import Counter

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [5]:
version = 'v7'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
source_dir = 'midi_npz'

out_dir = 'midi_encode/np/tstep'
continuous = True
short=False
def note_enc(note):
    if continuous: return note.continuous_repr(short=short)
    return note.duration_repr(short=short)

source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'midi_encode.csv'

In [7]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,9,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0


In [8]:
df_filtered = df.loc[df[source_dir].notna()]; df_filtered.shape

(38744, 25)

In [9]:
df_filter = (df_filtered.source != 'hooktheory') | (df_filtered.ht_time_signature == 4.0)
df_filtered = df_filtered.loc[df_filter]; df_filtered.shape

(37045, 25)

In [10]:
df_filtered = df_filtered.loc[df_filtered[f'{source_dir}_timesteps'] < 2e4]; df_filtered.shape

(37035, 25)

In [11]:
df_filtered[source_dir].values

array(['midi_npz/hooktheory/pianoroll/w/wayne-sharpe/yu-gi-oh-theme-song/chorus_key.npz',
       'midi_npz/hooktheory/pianoroll/w/what-a-day/kiefer/chorus_key.npz',
       'midi_npz/hooktheory/pianoroll/w/whiteflame/senbonzakura/verse_key.npz',
       'midi_npz/hooktheory/pianoroll/w/wham/last-christmas/intro_key.npz', ...,
       'midi_npz/from_mxl/wikifonia/Ernest Gold - Exodus Test.npz',
       'midi_npz/from_mxl/wikifonia/Traditional - JA, JA, SO BLAUW BLUHT DER ENZIAN.npz',
       "midi_npz/from_mxl/wikifonia/Andrae Crouch - The Blood Will Never Lose It's Power.npz",
       'midi_npz/from_mxl/wikifonia/Ross MacLean, Arthur Richardson - Too Fat Polka.npz'], dtype=object)

In [38]:
24+88

112

In [41]:
music21.note.Note('C7').octave

7

In [42]:
music21.note.Note(12).nameWithOctave

'C0'

In [45]:
music21.note.Note(12*8).nameWithOctave

'C7'

In [92]:
def get_max_notes(chordarr):
    notes_played = chordarr != 0
    note_range = chordarr[:,:,24:84]
    merged_inst = note_range[:,0:2].sum(axis=1) != 0
    return merged_inst.sum(axis=-1).max()

In [48]:
notes_played = chordarr != 0
note_range = chordarr[:,:,12:96]
merged_inst = note_range.sum(axis=1) != 0
merged_inst.sum(axis=-1).max()

8

In [56]:
notes_played.sum(axis=(1,2)).max()

14

In [55]:
merged_inst.sum(axis=-1)

array([0, 0, 0, 0, ..., 2, 2, 2, 0])

In [49]:
merged_inst.shape

(1697, 84)

In [47]:
get_max_notes(chordarr)

8

In [12]:
f = 'midi_npz/hooktheory/pianoroll/w/wayne-sharpe/yu-gi-oh-theme-song/chorus_key.npz'
chordarr = load_chordarr(version_path/f)

In [14]:
notes_played = chordarr != 0

In [36]:
notes_played.sum(axis=(1)).shape

(129, 127)

In [33]:
chordarr.shape

(1697, 14, 127)

In [12]:
seq = chordarr2seq(chordarr)

out_np = seq2np_duration(seq)

In [25]:
from fastprogress import progress_bar

In [31]:
??process_parallel

In [67]:
f2max = {}

In [93]:
def process_row(r):
    idx,row = r
    np_file = row[source_dir]
    chordarr = load_chordarr(version_path/np_file)
    m = get_max_notes(chordarr)
    return np_file,m

In [94]:
encoded_files = process_parallel(process_row, df_filtered.iterrows(), total=df_filtered.shape[0])

In [95]:
s = sorted(encoded_files.values())

In [96]:
s[-1000:]

[18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,


In [69]:
import numpy as np

In [70]:
s = sorted(f2max.values())

In [77]:
s[-5000:]

[12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,


In [73]:
np.histogram(list(f2max.values()))

(array([11373, 13494,  7236,  2685,  1733,   445,    52,    14,     1,     2]),
 array([ 1. ,  4.7,  8.4, 12.1, 15.8, 19.5, 23.2, 26.9, 30.6, 34.3, 38. ]))

In [ ]:
process_parallel()

In [13]:

VALTSEP = 12
VALTBOS = 13
PADDING_IDX = -1

In [14]:
# 4.
def npenc_func(n):
    return [n.pitch.pitchClass, n.pitch.octave, n.dur, n.inst]
    
def seq2np_duration(seq, note_func=npenc_func):
    "Note function returns a list of note components for spearation"
    result = [[VALTBOS, PADDING_IDX, PADDING_IDX, PADDING_IDX]]
    wait_count = 0
    for idx,timestep in enumerate(seq):
        flat_time = [note_func(n) for n in timestep if n.pitch.octave]
        if len(flat_time) == 0:
            wait_count += 1
        else:
            # pitch, octave, duration, instrument
            result.append([VALTSEP, PADDING_IDX, wait_count, PADDING_IDX])
            result.extend(flat_time)
            wait_count = 0
    return np.array(result, dtype=int)

In [15]:
def transform_func(file, out_file, row):
    chordarr = load_chordarr(file)
    seq = chordarr2seq(chordarr)
    npenc = seq2np_duration(seq, npenc_func)
    np.save(out_file, npenc)

In [16]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension='.npy'
       )

In [17]:
def npenc2seq(npenc):
    seq = []
    tstep = []
    for x in npenc:
        n,o,d,i = x
        if n == VALTSEP: 
            if len(tstep) > 0: seq.append(tstep)
            tstep = []
            for i in range(d): seq.append([])
        else:
            tstep.append(NoteEnc(n+((o+1)*12),d,i))
    return seq

In [18]:
# seq_back = npenc2seq(out_np)
# seq_back
# seq

In [20]:
# for r in df_filtered.iterrows():
#     parallel_func(r)
#     break

In [21]:
encoded_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

In [22]:
tdf = pd.DataFrame(data={out_dir: list(encoded_files.values())}, index=list(encoded_files.keys()))

In [23]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

((37035, 1), (39942, 25), (39942, 26))

In [24]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps,midi_encode/np/dur
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0,NaN
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0,midi_encode/np/dur/hooktheory/pianoroll/w/wayn...
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0,midi_encode/np/dur/hooktheory/pianoroll/w/what...
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0,midi_encode/np/dur/hooktheory/pianoroll/w/whit...
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0,midi_encode/np/dur/hooktheory/pianoroll/w/wham...
